# Numba
## Numba 介绍
Numba 是一个开源的、NumPy 感知的 Python JIT (Just-In-Time) 编译器，它能将 Python 函数（尤其是包含大量数学运算、NumPy 操作和循环的函数）转换为优化的机器码，从而在运行时显著提高其执行速度，有时甚至能达到与 C 或 Fortran 语言相近的性能。

核心特点和优势：
- 即时编译 (JIT)
    - Numba 在函数首次被调用时对其进行编译。这意味着初始调用可能会有一些延迟（编译开销），但后续的调用将直接运行已编译的、高度优化的机器码。
它不需要开发者编写 C/C++ 代码或学习复杂的编译工具链。
- NumPy 感知:
    - 擅长优化使用了 NumPy 数组和通用函数 (ufuncs) 的代码。生成高效的循环和内存访问模式。
- 装饰器驱动:
    - Numba 的主要使用方式是通过装饰器（如 @jit, @njit, @vectorize, @guvectorize）来标记需要编译的 Python 函数。
- 多种编译模式:
    - nopython 模式：这是 Numba 性能最高的模式。在此模式下，Numba 会将整个函数编译为不依赖 Python C API 的机器码。
        - 如果函数中包含 Numba 无法处理的 Python 特性，编译会失败。
        - @njit 是 @jit(nopython=True) 的简写。
    - object 模式：如果 nopython 模式编译失败，Numba 可以回退到对象模式。
        - 在此模式下，Numba 会编译它能处理的部分，而不能处理的部分则通过 Python C API 来执行。
        - 性能提升通常不如 nopython 模式显著，但能让更多种类的 Python 代码运行。
    - loop-lifting：在对象模式下，Numba 仍可能识别并优化内部的循环，即使函数的其他部分不能完全编译。
- 并行化 (Parallelism)
    - 在 @jit 或 @njit 中设置 parallel=True，Numba 可以尝试自动并行化代码中的某些循环（通常是 NumPy 操作或使用 numba.prange 的循环），利用多核 CPU 进一步提升性能。
- GPU 支持:
    - Numba 也支持为 NVIDIA CUDA 和 AMD ROCm GPU 编译 Python 代码，使得用 Python 编写高性能的 GPU 内核成为可能。

适用场景：
- 计算密集型的数值算法。
- 涉及大型 NumPy 数组操作的代码。
- 包含大量 Python 循环且循环体是数值计算的场景。
- 需要将 Python 原型代码快速转换为高性能代码而不想重写为 C/C++ 的情况。

不适用或效果不佳的场景：
- 主要涉及字符串操作、I/O 操作、或大量使用 Python 动态特性和复杂数据结构（如 Pandas DataFrame 的高层操作，虽然 Numba 可以优化其底层的 NumPy 操作）的代码。Numba 的设计目标是加速数值计算。
- 函数调用开销本身远大于函数内部计算量的短小函数（除非这些函数被大量重复调用）。

## 函数 `_align_index_to_nb`
```python
@njit(cache=True)
def _align_index_to_nb(a: tp.Array1d, b: tp.Array1d) -> tp.Array1d:
    idxs = np.empty(b.shape[0], dtype=np.int64)
    g = 0
    for i in range(b.shape[0]):
        for j in range(a.shape[0]):
            if b[i] == a[j]:
                idxs[g] = j
                g += 1
                break
    return idxs
```
`@njit(cache=True)` 是什么意思？
- `jit` 是 Numba 的核心 JIT 编译装饰器。`njit` 是 Numba 提供的一个装饰器，是 `jit(nopython=True)` 的别名。
    - `nopython=True` 告诉 Numba 必须以 `nopython` 模式编译函数。
        - 在此模式下，Numba 会尝试将整个函数完全编译成机器码，不依赖任何 Python C API。
- `cache=True` 启用了 Numba 的编译缓存功能。
    - 作用：Numba 在第一次成功编译一个函数（及其特定参数类型签名）后，会将其编译结果（通常是 LLVM 生成的机器码或其表示）存储到文件系统中。
    - 可以减少后续运行的编译时间：在后续运行相同的 Python 脚本（或导入该模块的其他脚本）时，如果该函数的源代码、Numba 版本、Python 版本以及相关的环境变量（如 CPU 特性）没有发生变化，Numba 会直接从缓存中加载已编译的版本，从而跳过耗时的编译过程。
    - 缓存机制
        - Numba 会检查源文件（.py 文件）的修改时间戳。如果源文件被修改，缓存通常会失效，函数会被重新编译。
        - 缓存与函数的签名（参数类型）相关。对于同一个函数，如果使用不同类型的参数调用它，Numba 会为每种类型组合编译一个专门的版本，并分别缓存。